# Чтение данных из файла

In [14]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import DatetimeTickFormatter, ColumnDataSource
from bokeh.palettes import Spectral11
import pandas as pd
import os

# Read data (by column):
path = os.path.abspath(os.path.dirname(os.path.dirname('__file__')))
file = os.path.join(path, 'weight_2.csv')

data = pd.read_csv(file)
data.head(6)

#Define parameters
t = pd.to_numeric(data['T'])
x = pd.to_numeric(data['X'])
#y = pd.to_numeric(data['Y'])
#z = pd.to_numeric(data['Z'])

#Take data  and present in a graph
output_notebook()

p = figure(plot_width=900,plot_height=400,tools='pan,wheel_zoom,box_zoom,reset,save',active_scroll='wheel_zoom',active_drag='pan')
p.line(t, x, line_width=2, color=Spectral11[0], legend_label='ax')
#p.line(t, y, line_width=2, color=Spectral11[1], legend_label='ay')
#p.line(t, z, line_width=2, color=Spectral11[2], legend_label='az')

p.yaxis.axis_label = 'Amplitude'
p.xaxis.axis_label = 'Time [ms]'
'''
p.xaxis.formatter=DatetimeTickFormatter(
    milliseconds=["%S.%f"],
    seconds=["%S.%f"],
    minsec=["%M:%S"],
    minutes=["%M"],
    hourmin=["%H:%M"],
    hours=["%H"],
    days=["%Y-%m-%d"],
    months=["%Y-%m"],
    years=["%Y"],
)
'''
show(p)

Loading BokehJS ...

# Линейная интерполяция

Цель: получить равномерный шаг по оси времени *t*.

Решение: узловые точки соединяются отрезками прямых, то есть через каждые две точки $(t_i, f_i)$ и $(t_{i+1}, f_{i+1})$ проводится полином первой степени:

\begin{align}
\frac{f - f_0}{t - t_0} = \frac{f_1 - f_0}{t_1 - t_0},
\end{align}

Отрезок времени *t* разбивается равномерно на N точек и в каждой из этих точек вычисляется функция:

\begin{align}
f = f_0 + (t-t_0)\frac{f_1 - f_0}{t_1-t_0} = \frac{f_0(t_1-t)+f_1(t-t_0)}{t_1-t_0} = f_0 \left(1 - \frac{t - t_0}{t_1 - t_0}\right) + f_1 \left(\frac{t - t_0}{t_1 - t_0}\right)
\end{align}

* [Oдномерная кусочно-линейная интерполяция функции](https://pyprog.pro/mathematical_functions/interp.html)
* [numpy.interp](https://docs.scipy.org/doc/numpy/reference/generated/numpy.interp.html)


In [8]:
from scipy import interpolate
import numpy as np

ti = np.arange(t[0], t[len(t)-1], 0.0001)
xi = np.interp(ti, t, x)
#yi = np.interp(ti, t, y)
#zi = np.interp(ti, t, z)

f = figure(plot_width=900,plot_height=400,tools='pan,wheel_zoom,box_zoom,reset,save',active_scroll='wheel_zoom',active_drag='pan')
f.line(ti, xi, line_width=2, color='red', legend_label='ax')
#f.line(ti, yi, line_width=2, color=Spectral11[1], legend_label='ax')
#f.line(ti, xi, line_width=2, color=Spectral11[2], legend_label='ax')
f.yaxis.axis_label = 'Amplitude'
f.xaxis.axis_label = 'Time [s]'
show(f)

# Преобразование Фурье

* [Fourier transform - 1](https://www.ritchievink.com/blog/2017/04/23/understanding-the-fourier-transform-by-example/)
* [Fourier transform - 2](https://eax.me/python-numpy/)


In [10]:
xa = np.fft.fft(xi)
#ya = np.fft.fft(yi)
#za = np.fft.fft(zi)
T = 0.0001  # sampling interval 
N = xi.size
# 1/T = frequency
f = np.linspace(0, 1 / T, N)

ff = figure(plot_width=900,plot_height=400,tools='pan,wheel_zoom,box_zoom,reset,save',active_scroll='wheel_zoom',active_drag='pan')
ff.vbar(x=f[:N // 2], top=np.abs(xa)[:N // 2] * 1 / N, width=0.5)  # 1 / N is a normalization factor
#ff.vbar(x=f[:N // 2], top=np.abs(ya)[:N // 2] * 1 / N, width=0.1)  # 1 / N is a normalization factor
#ff.vbar(x=f[:N // 2], top=np.abs(za)[:N // 2] * 1 / N, width=0.1)  # 1 / N is a normalization factor
#ff.line(f[:N // 2], np.abs(xa)[:N // 2] * 1 / N, line_width=2, color=Spectral11[0], legend_label='ax')
#ff.line(f[:N // 2], np.abs(ya)[:N // 2] * 1 / N, line_width=1, color=Spectral11[1], legend_label='yx')
#ff.line(f[:N // 2], np.abs(za)[:N // 2] * 1 / N, line_width=1, color=Spectral11[2], legend_label='zx')
ff.yaxis.axis_label = 'Amplitude'
ff.xaxis.axis_label = 'Frequency [Hz]'
show(ff)